In [3]:
import numpy as np
import pandas as pd
import os
import glob
import re
import nilearn

from nilearn import plotting
from nilearn.signal import clean
import logging


In [34]:
path_derivatives = '../data/derivatives'

path_halfpipe_timeseries = '{}/halfpipe'.format(path_derivatives)
path_fmriprep_confounds = '{}/fmriprep'.format(path_derivatives)

path_atlas = '../data/atlas'

name_atlas = 'schaefer400' ### doit etre aligné avec le nom du fichier de labels, tout dois etre dans path_atlas

output_path = 'draft/output'

task = 'task-rest' ### faire une boucle?


In [5]:
### il faudra que les noms soient standardisés

FEATURE_RENAME_MAP = {
    "baseline": "baseline",
    "corrMatrix1": "compcor",
    "corrMatrix4": "simple",
    "corrMatrix5": "simple+gsr",
    "corrMatrix3": "scrubbing.5+gsr",
    "corrMatrix2": "scrubbing.5",
}


In [6]:
path_label_schaefer = '{}/atlas-Schaefer2018Combined_dseg.tsv'.format(path_atlas)

label_schaefer = list(pd.read_csv(path_label_schaefer, sep = '\t', header = None)[1])
print(label_schaefer)


['Schaefer2018_17Networks_LH_VisCent_ExStr_1', 'Schaefer2018_17Networks_LH_VisCent_ExStr_2', 'Schaefer2018_17Networks_LH_VisCent_ExStr_3', 'Schaefer2018_17Networks_LH_VisCent_ExStr_4', 'Schaefer2018_17Networks_LH_VisCent_ExStr_5', 'Schaefer2018_17Networks_LH_VisCent_ExStr_6', 'Schaefer2018_17Networks_LH_VisCent_Striate_1', 'Schaefer2018_17Networks_LH_VisCent_ExStr_8', 'Schaefer2018_17Networks_LH_VisCent_ExStr_9', 'Schaefer2018_17Networks_LH_VisCent_ExStr_10', 'Schaefer2018_17Networks_LH_VisCent_ExStr_11', 'Schaefer2018_17Networks_LH_VisCent_ExStr_12', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_1', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_2', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_3', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_4', 'Schaefer2018_17Networks_LH_VisPeri_ExStrInf_5', 'Schaefer2018_17Networks_LH_VisPeri_StriCal_1', 'Schaefer2018_17Networks_LH_VisPeri_StriCal_2', 'Schaefer2018_17Networks_LH_VisPeri_ExStrSup_8', 'Schaefer2018_17Networks_LH_VisPeri_ExStrSup_9', 'Sc

In [7]:
len(label_schaefer)


434

In [8]:
sujets = []

for nom in os.listdir(path_halfpipe_timeseries,):
    chemin_complet = os.path.join(path_halfpipe_timeseries, nom)
    if os.path.isdir(path_halfpipe_timeseries,):
        sujets.append(nom)


In [9]:
num_run = '1'


In [10]:
strategies = ['corrMatrix1', 'corrMatrix2', 'corrMatrix3', 'corrMatrix4', 'corrMatrix5']


In [11]:
strategy = strategies[0]


In [ ]:
## Cedric work to get all time series

def get_all_halfpipe_timeseries_paths() -> list:
    """Extract all time series TSV path from input directory"""
    INPUT_DIRECTORY = '../data/'
    timeseries_path_pattern = os.path.join(INPUT_DIRECTORY, '**', '*_timeseries.tsv')
    return glob.glob(timeseries_path_pattern, recursive=True)

def extract_timeseries_metadata(files_paths:list) -> tuple:
    """Extract metadata from the file path and return a tuple of subject, session, run, strategy, and atlas"""
    subjects, sessions, runs, strategies, atlases = [], [], [], [], []
    for file_path in files_paths:
        subject_match = re.search(r'/(?:sub-([^/]+))', file_path)
        session_match = re.search(r'/(?:ses-([^/]+))', file_path)
        run_match = re.search(r'/(?:run-([^/]+))', file_path)
        strategy_match = re.search(r'(?:feature-([^_]+))', file_path)
        atlas_match = re.search(r'(?:atlas-([^_]+))', file_path)

        subject = subject_match.group(1) if subject_match else None
        session = session_match.group(1) if session_match else None
        run = run_match.group(1) if run_match else None
        strategy = strategy_match.group(1) if strategy_match else None
        atlas = atlas_match.group(1) if atlas_match else None

        if subject: subjects.append(subject)
        if session: sessions.append(session)
        if run: runs.append(run)
        if strategy: strategies.append(strategy)
        if atlas: atlases.append(atlas)
        
    subjects = list(set(subjects)) 
    sessions = list(set(sessions))
    runs = list(set(runs))
    strategies = list(set(strategies))
    atlases = list(set(atlases))
    return subjects, sessions, runs, strategies, atlases


time_series_files_paths = get_all_halfpipe_timeseries_paths()
subjects, sessions, runs, strategies, atlases = extract_timeseries_metadata(time_series_files_paths)
print(subjects)
print(sessions)
print(runs)
print(strategies) 
print(atlases)


['10225', '10228', '10206', '10189', '10227', '10217', '10249', '10235', '10171', '10159']
[]
[]
['corrMatrix2', 'corrMatrix5', 'corrMatrix4', 'corrMatrix1', 'corrMatrix3']
['schaefer400']


In [55]:
dict_timeseries = {}
missing_list = []
dict_mean_framewise = {}
dict_scrubvolume = {}

for sub in sujets:
    
    
    
    # timeseries = '{}/{}/func/task-rest/{}_task-rest_run-{}_feature-{}_atlas-{}_timeseries.tsv'.format(
    #     path_halfpipe_timeseries, sub, sub, num_run, strategy, name_atlas)
    
    timeseries = '{}/{}/func/task-rest/{}_{}_feature-{}_atlas-{}_timeseries.tsv'.format(
        path_halfpipe_timeseries, sub, sub, task, strategy, name_atlas)
    confounds = '{}/{}/func/{}_{}_desc-confounds_timeseries.tsv'.format(
        path_fmriprep_confounds, sub, sub, task)
    
    try:
        df_timeseries = pd.read_csv(timeseries, sep='\t', header=None)
        df_timeseries.columns = label_schaefer
        dict_timeseries[sub] = df_timeseries

        # TODO : later add the confounds in clean code
        df_confounds = pd.read_csv(confounds, sep='\t')
        mean_framewise = df_confounds['framewise_displacement'].mean()
        dict_mean_framewise[sub] = mean_framewise

        n_motion_outliers = df_confounds.filter(like='motion_outlier').shape[1]
        dict_scrubvolume[sub] = n_motion_outliers
 


        


    except FileNotFoundError:
        missing_list.append(sub)
        continue


In [14]:
len(missing_list)


0

In [ ]:
# Initialisation du compteur de NaNs pour chaque ROI
nan_counts = {label: 0 for label in label_schaefer}
total_subjects = len(dict_timeseries)

# Parcourir tous les sujets
for df in dict_timeseries.values():
    for label in label_schaefer:
        if label in df.columns and df[label].isna().all():
            nan_counts[label] += 1

# Création du DataFrame
df_nan_prop = pd.DataFrame({
    "ROI": list(nan_counts.keys()),
    "proportion_nan": [nan_counts[label] / total_subjects for label in label_schaefer]
})


In [16]:
df_nan_prop.sort_values(by='proportion_nan', ascending=False).reset_index(drop=True)[:30]


,ROI,proportion_nan
0,Buckner2011_17Networks_1,1.0
1,Buckner2011_17Networks_14,1.0
2,Schaefer2018_17Networks_LH_Limbic_TempPole_4,1.0
3,Schaefer2018_17Networks_LH_Limbic_TempPole_1,0.9
4,Schaefer2018_17Networks_RH_Limbic_TempPole_2,0.8
5,Schaefer2018_17Networks_RH_Limbic_TempPole_6,0.8
6,Schaefer2018_17Networks_RH_Limbic_OFC_5,0.8
7,Schaefer2018_17Networks_LH_Limbic_TempPole_2,0.8
8,Schaefer2018_17Networks_RH_Limbic_TempPole_1,0.8
9,Schaefer2018_17Networks_RH_Limbic_TempPole_3,0.7


In [17]:
# Identifier les labels avec une proportion de NaN supérieure à 0.5
labels_a_supprimer = df_nan_prop[df_nan_prop["proportion_nan"] > 0.5]["ROI"].tolist()

# Supprimer ces labels de chaque DataFrame dans dict_timeseries
for key, df in dict_timeseries.items():
    dict_timeseries[key] = df.drop(columns=labels_a_supprimer, errors='ignore')


In [18]:
# Initialiser une liste pour stocker les résultats
liste_proportions = []

# Parcourir chaque sujet et son DataFrame associé
for sujet, df in dict_timeseries.items():
    # Calculer le nombre total de ROI
    total_roi = df.shape[1]
    
    # Compter le nombre de ROI contenant au moins un NaN
    roi_nan = df.isna().any().sum()
    
    # Calculer la proportion de ROI avec des NaN
    proportion_nan = roi_nan / total_roi if total_roi > 0 else 0
    
    # Ajouter les résultats à la liste
    liste_proportions.append({'sujet': sujet, 'proportion_nan': proportion_nan})

# Créer le DataFrame final
df_proportion_nan = pd.DataFrame(liste_proportions)


In [19]:
df_proportion_nan.sort_values(by='proportion_nan', ascending=False).reset_index(drop=True)


,sujet,proportion_nan
0,sub-10217,0.028777
1,sub-10159,0.028777
2,sub-10228,0.028777
3,sub-10171,0.028777
4,sub-10227,0.023981
5,sub-10249,0.019185
6,sub-10206,0.011990
7,sub-10235,0.007194
8,sub-10189,0.007194
9,sub-10225,0.004796


In [ ]:
#!TODO: clean code stopped here 

def impute_and_clean(df):
    # Étape 1 : Remplacer les NaN par la moyenne de leur ligne (mean sur l'axe 1)
    row_means = df.mean(axis=1, skipna=True)
    df_filled = df.T.fillna(row_means).T  # on transpose pour appliquer sur les lignes, puis on retranspose

    # Étape 2 : Vérification (optionnel mais utile pour logger ce qu'on fait)
    if df_filled.isna().any().any():
        logging.warning("Certaines valeurs n'ont pas pu être imputées par la moyenne de ligne.")

    # Étape 3 : Nettoyage avec nilearn (standardisation + détrend)
    cleaned = clean(df_filled.values, detrend=True, standardize='zscore_sample')

    # Étape 4 : Retour sous forme de DataFrame
    return pd.DataFrame(cleaned, columns=df.columns, index=df.index)


In [21]:
dict_clean_timeseries = {}
for sub in dict_timeseries.keys():
    dict_clean_timeseries[sub] = impute_and_clean(dict_timeseries[sub])


In [22]:
dict_clean_timeseries[sub]


,Schaefer2018_17Networks_LH_VisCent_ExStr_1,Schaefer2018_17Networks_LH_VisCent_ExStr_2,Schaefer2018_17Networks_LH_VisCent_ExStr_3,Schaefer2018_17Networks_LH_VisCent_ExStr_4,Schaefer2018_17Networks_LH_VisCent_ExStr_5,Schaefer2018_17Networks_LH_VisCent_ExStr_6,Schaefer2018_17Networks_LH_VisCent_Striate_1,Schaefer2018_17Networks_LH_VisCent_ExStr_8,Schaefer2018_17Networks_LH_VisCent_ExStr_9,Schaefer2018_17Networks_LH_VisCent_ExStr_10,...,Buckner2011_17Networks_7,Buckner2011_17Networks_8,Buckner2011_17Networks_9,Buckner2011_17Networks_10,Buckner2011_17Networks_11,Buckner2011_17Networks_12,Buckner2011_17Networks_13,Buckner2011_17Networks_15,Buckner2011_17Networks_16,Buckner2011_17Networks_17
0,-1.269069,-1.366775,-1.232921,-0.591805,-0.646051,-1.025877,-0.392936,-0.034379,-0.571169,-1.167148,...,0.135191,0.197914,1.249865,-0.323781,1.018939,1.083037,1.555505,0.687387,2.000476,0.918438
1,-1.233519,-0.832193,-0.970351,0.496777,0.276453,-0.898063,0.012422,-0.640618,-0.693913,-1.023075,...,-0.156975,0.156124,0.999426,-0.632410,0.747845,1.089863,1.273734,0.941834,1.942699,1.389675
2,-0.492140,-0.180654,-0.281531,0.496793,-0.108141,-0.865157,0.261295,-0.177849,-0.682845,-1.400336,...,0.196687,0.700435,0.454992,1.294984,0.980150,1.636779,1.760936,0.052958,1.708853,1.598759
3,0.394993,0.745206,-0.675988,1.289641,-0.190020,-0.321265,0.015653,0.125472,-0.564733,-1.169269,...,0.648794,1.296469,0.973884,-0.515828,0.972485,1.414950,1.639898,1.345068,2.476335,1.759630
4,-0.260036,-0.528143,-0.728200,0.063813,-0.280744,-0.507732,-0.846634,-0.412376,-0.509810,-0.723055,...,0.329472,1.236851,0.525557,0.330826,0.797199,0.982028,1.792587,0.753899,2.419462,2.182420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,-0.421478,-1.618184,-0.161755,0.065262,0.569468,0.087091,-0.249106,1.059227,-0.213843,-1.122131,...,-0.218789,-0.424957,-0.000277,2.082095,-1.542730,0.442567,1.336581,-0.389693,0.400878,0.812836
148,-1.494513,-0.760332,-1.047642,-1.157972,-1.338034,-1.035745,-0.195441,-0.012604,-1.089376,-0.576170,...,-0.849502,-1.377169,-1.092836,-0.280160,-0.787743,-0.724574,-1.712581,-1.286585,-1.437270,-0.656553
149,-1.385900,-0.731730,0.597515,-0.864572,-1.391119,-1.773529,-1.410215,0.023589,-1.733685,-1.981569,...,-0.852835,-1.417107,-0.502684,0.718001,-0.933663,-0.816033,-1.453257,-0.475802,-1.113476,-0.593475
150,-1.527978,-0.363107,-0.706269,-0.507213,0.186148,-0.641976,-0.368638,-0.298063,-0.226714,-0.203344,...,-0.161793,-0.760561,-0.025585,0.568242,-0.134573,0.353101,0.082910,-0.636427,-0.255939,0.875491


In [23]:
volume_path = '{}/atlas-Schaefer2018Combined_dseg.nii.gz'.format(path_atlas)

coords = plotting.find_parcellation_cut_coords(volume_path)


In [24]:
coords.shape


(434, 3)

In [25]:
df_coords = pd.DataFrame(coords, index=label_schaefer, columns=['x', 'y', 'z'])


In [26]:
df_coords


,x,y,z
Schaefer2018_17Networks_LH_VisCent_ExStr_1,-37.286885,-62.519672,-17.122951
Schaefer2018_17Networks_LH_VisCent_ExStr_2,-24.904651,-73.476744,-9.988372
Schaefer2018_17Networks_LH_VisCent_ExStr_3,-38.226496,-81.685185,-15.673789
Schaefer2018_17Networks_LH_VisCent_ExStr_4,-18.255814,-87.476744,-14.587209
Schaefer2018_17Networks_LH_VisCent_ExStr_5,-26.312613,-97.649550,-11.372072
...,...,...,...
Buckner2011_17Networks_13,-34.875911,-70.730917,-36.717108
Buckner2011_17Networks_14,-21.300000,-73.700000,-39.700000
Buckner2011_17Networks_15,-16.932432,-39.283784,-45.918919
Buckner2011_17Networks_16,-7.010251,-52.796128,-44.422551


In [27]:
len(labels_a_supprimer)


17

In [28]:
df_coords = df_coords[~df_coords.index.isin(labels_a_supprimer)]


In [29]:
remaining_labels = list(set(label_schaefer) - set(labels_a_supprimer))


In [ ]:
dict_corrpearson_matrix = {}
for sub in dict_clean_timeseries.keys():
    dict_corrpearson_matrix[sub] = dict_clean_timeseries[sub].corr(method='pearson')


In [ ]:
name_ds = 'halfpipe_outputs_timeseries_HBN_dev'
fmriprep_version = 'fmriprep-25.0.0'
name_atlas = 'atlas-schaefercombined'
task_name = 'task-rest'
space = 'space-MNI152NLin2009cAsym'

nroi = dict_clean_timeseries[sub].shape[1]

strat = FEATURE_RENAME_MAP.get(strategy, strategy)

output_folder_path = '{}/{}/func'.format(output_path, sub)

"sub-1_ses-timepoint1_task-probabilisticclassification_run-01_seg-MIST7_desc-denoiseSimple_timeseries.tsv"
"sub-1_ses-timepoint1_task-probabilisticclassification_run-01_seg-MIST7_meas-PearsonCorrelation_desc-denoiseSimple_relmat.tsv"


In [ ]:
for sub in dict_clean_timeseries.keys():

    ### timeseries
    file_name = '{}_{}_seg-{}{}_{}_{}_desc-denoise{}_timeseries.tsv'.format(sub, task_name, name_atlas, nroi, strat)
    file_output = '{}/{}'.format(output_folder_path, file_name)
    os.makedirs(os.path.dirname(file_output), exist_ok=True)
    
    dict_clean_timeseries[sub].columns = range(dict_clean_timeseries[sub].shape[1])
    dict_clean_timeseries[sub].to_csv(file_output, sep="\t", header=True, index=False)

    ### connectomes pearson
    file_name = '{}_{}_seg-{}{}_{}_{}_meas-PearsonCorrelation_desc-denoise{}_relmat.tsv'.format(sub, task_name, name_atlas, nroi, strat)
    file_output = '{}/{}'.format(output_folder_path, file_name)
    os.makedirs(os.path.dirname(file_output), exist_ok=True)
    
    dict_corrpearson_matrix[sub].columns = range(dict_corrpearson_matrix[sub].shape[1])
    dict_corrpearson_matrix[sub].to_csv(file_output, sep="\t", header=True, index=False)

    ### json timeseries
    


In [103]:
json_meas_PearsonCorrelation_relmat = {
    "Measure": "Pearson correlation",
    "MeasureDescription": "Pearson correlation",
    "Weighted": False,
    "Directed": False,
    "ValidDiagonal": True,
    "StorageFormat": "Full",
    "NonNegative": "",
    "Code": "https://github.com/pbergeret12/HalfPipe2Bids/tree/main"
}


In [ ]:
json_dataset_description = {
    "BIDSVersion": "1.9.0",
    "License": None,
    "Name": None,
    "ReferencesAndLinks": [],
    "DatasetDOI": None,
    "DatasetType": "derivative",
    "GeneratedBy": [
        {
            "Name": "Halfpipe2Bids",
            "Version": "0.1",
            "CodeURL": "https://github.com/pbergeret12/HalfPipe2Bids/tree/main"
        }
    ],
    "HowToAcknowledge": "Please refer to our repository: https://github.com/pbergeret12/HalfPipe2Bids/tree/main."
}
